In [1]:
import json
import pickle
from argparse import ArgumentParser, Namespace
from pathlib import Path
from typing import Dict

import torch
from torch.utils.data import DataLoader
from tqdm import trange
from model import SeqClassifier
from dataset import SeqClsDataset
from utils import Vocab

TRAIN = "train"
DEV = "eval"
SPLITS = [TRAIN, DEV]

In [2]:
cache_dir = Path("./cache/intent/")
data_dir = Path("./data/intent/")
batch_size = 128
max_len = 128
with open(cache_dir / "vocab.pkl", "rb") as f:
    vocab: Vocab = pickle.load(f)

intent_idx_path = cache_dir / "intent2idx.json"
intent2idx: Dict[str, int] = json.loads(intent_idx_path.read_text())

data_paths = {
    split: data_dir / f"{split}.json" 
    for split in SPLITS
}
data = {
    split: json.loads(path.read_text()) 
    for split, path in data_paths.items()
}
datasets: Dict[str, SeqClsDataset] = {
    split: SeqClsDataset(split_data, vocab, intent2idx, max_len)
    for split, split_data in data.items()
}
# TODO: crecate DataLoader for train / dev datasets
train_data_loader = DataLoader(
    datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=datasets["train"].collate_fn,
)
dev_data_loader = DataLoader(
    datasets["eval"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=datasets["eval"].collate_fn,
)


In [3]:
import numpy as np

embeddings = torch.load(cache_dir / "embeddings.pt")
num_layers = 2
hidden_size = 512
dropout = 0.1
bidirectional = 1
device = "cuda"
num_epoch = 50
lr = 1e-3
# TODO: init model and move model to target device(cpu / gpu)
model = SeqClassifier(
    embeddings,
    int(hidden_size),
    int(num_layers),
    float(dropout),
    int(bidirectional),
    150,
)
model.to(device)

# TODO: init optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.CrossEntropyLoss()

epoch_pbar = trange(num_epoch, desc="Epoch")
train_losses = []
train_accs = []
valid_losses = []
valid_accs = []
best_valid_loss = float('inf')
for epoch in epoch_pbar:
    # TODO: Training loop - iterate over train dataloader and update model weights
    model.train()
    epoch_train_losses = []
    epoch_train_accs = []
    for batch in train_data_loader:
        ids = batch["ids"].to(device)
        labels = batch["labels"].to(device)
        pred = model(ids).to(device)
        loss = criterion(pred, labels).to(device)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_train_losses.append(loss.item())
        epoch_train_accs.append((pred.argmax(dim=1) == labels).float().mean().item())

    # TODO: Evaluation loop - calculate accuracy and save model weights
    model.eval()
    epoch_eval_losses = []
    epoch_eval_accs = []
    with torch.no_grad():
        for batch in dev_data_loader:
            ids = batch["ids"].to(device)
            labels = batch["labels"].to(device)
            pred = model(ids).to(device)
            loss = criterion(pred, labels).to(device)
            epoch_eval_losses.append(loss.item())
            epoch_eval_accs.append((pred.argmax(dim=1) == labels).float().mean().item())
            
    train_losses.extend(epoch_train_losses)
    train_accs.extend(epoch_train_accs)
    valid_losses.extend(epoch_eval_losses)
    valid_accs.extend(epoch_eval_accs)
    
    epoch_train_loss = np.mean(epoch_train_losses)
    epoch_train_acc = np.mean(epoch_train_accs)
    epoch_valid_loss = np.mean(epoch_eval_losses)
    epoch_valid_acc = np.mean(epoch_eval_accs)
    
    print(epoch_train_acc)
    print(epoch_valid_acc)
    if epoch_valid_loss < best_valid_loss:
        best_valid_loss = epoch_valid_loss
        torch.save(model.state_dict(), 'lstm.pt')

# TODO: Inference on test set

Epoch:   2%|███▎                                                                                                                                                                    | 1/50 [00:25<20:39, 25.30s/it]

0.2686926202248719
0.6127232164144516


Epoch:   4%|██████▋                                                                                                                                                                 | 2/50 [00:50<20:24, 25.51s/it]

0.8064971753096176
0.8203590040405592


Epoch:   6%|██████████                                                                                                                                                              | 3/50 [01:16<20:08, 25.71s/it]

...

1.0
0.9202938998738924


Epoch:  98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋   | 49/50 [21:56<00:26, 26.25s/it]

1.0
0.9185267885526022


Epoch: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [22:23<00:00, 26.86s/it]

1.0
0.9206194207072258


In [4]:
for batch in train_data_loader:
    ids = batch["ids"].to(device)
    labels = batch["labels"].to(device)
    pred = model(ids).to(device)
    print(labels)
    print(pred.argmax(dim=1))
    break

tensor([118,  39,  21,  42, 144,  10, 100,  39,  41, 140, 122, 129,  97, 146,
         72,  35,  71, 115,  81,  58,  84,  74, 102,  83, 117,  16,  47,  89,
         79,  93, 149,  57,  19, 113,  75,  42, 145, 121,  31, 120,  95, 126,
        112,  20,  88,  50, 136, 114, 106, 127,  81, 145, 108,  52,  21, 100,
         76, 111,  73,  40,  20,   9, 103,  34, 133,  82,  14,  64,   7,  17,
        101,  15,  52, 139,  99, 114, 102,  25, 111, 141,  67, 122,  14,  20,
         48, 121,  81,  18, 113, 101,   5,  47,   6,  87,  94, 133, 120,  60,
         37,  26, 110,  31,  47, 120, 115,   1, 112,  63,  59, 110,  45,  92,
        117, 139,  95,  40,  44,  95, 116,  24,  32,   4,  16, 138,  17,  50,
         12,  32], device='cuda:0')
tensor([118,  39,  21,  42, 144,  10, 100,  39,  41, 140, 122, 129,  97, 146,
         72,  35,  71, 115,  81,  58,  84,  74, 102,  83, 117,  16,  47,  89,
         79,  93, 149,  57,  19, 113,  75,  42, 145, 121,  31, 120,  95, 126,
        112,  20,  88,  50, 

In [19]:
a = torch.tensor([[1, 2, 3], [4, 5, 6]])
a.shape

torch.Size([2, 3])

In [20]:
a

tensor([[1, 2, 3],
        [4, 5, 6]])

In [21]:
a.argmax(dim=1)

tensor([2, 2])

In [4]:
embeddings = torch.load(cache_dir / "embeddings.pt")
num_layers = 2
hidden_size = 512
dropout = 0.1
bidirectional = 1
device = "cuda"
num_epoch = 50
lr = 1e-3

# load pytorch model
lstm_model = SeqClassifier(
    embeddings,
    int(hidden_size),
    int(num_layers),
    float(dropout),
    int(bidirectional),
    150,
)
lstm_model.load_state_dict(torch.load("lstm.pt"))
lstm_model.to(device)
lstm_model.eval()
for batch in train_data_loader:
    ids = batch["ids"].to(device)
    labels = batch["labels"].to(device)
    pred = lstm_model(ids).to(device)
    print(labels)
    print(pred.argmax(dim=1))
    break

tensor([ 90,  76,   4,  14,  19,  30, 136,  76,  44,  99,  88,  33, 128,  87,
         59,  80, 112,  39,   9,  82,  14, 145,  33,  33,  90,  79,  94,   4,
        119, 135,  94,  67,  29,   3,  30,  87, 114,  49,   8,  91,   7,  36,
         86, 100,  63, 137, 142, 102,  37,  39,  43,  69,  91, 118,  43,   8,
        140, 132,  23,  44,  14,  88,  19,   1, 135,  72, 126,  23, 120,  42,
         69,  89, 121,  53,   9,  14,  20,  19, 101,  50,  20, 105,  49,  17,
         83, 121, 129,  34, 121,  23,  94,  56,  14, 139,  97,   3,  78,  27,
        147, 113, 144, 124,  67,  56,  41,  95, 138, 107,   5, 110,   8, 140,
          7,  67,  83,  45,  68,   0, 135,  57, 147,  22, 112,  82,  15,  95,
         66, 132], device='cuda:0')
tensor([ 90,  76,   4,  14,  19,  30, 136,  76,  44,  99,  88,  33, 128,  87,
         59,  80, 112,  39,   9,  82,  14, 145,  33,  33,  90,  79,  94,   4,
        119, 135,  94,  67,  29,   3,  30,  87, 114,  49,   8,  91,   7,  36,
         86, 100,  63, 137, 

In [6]:
type(lstm_model)

collections.OrderedDict

In [5]:
torch.load("lstm.pt").__dict__

{'_metadata': OrderedDict([('', {'version': 1}),
              ('embed', {'version': 1}),
              ('rnn', {'version': 1}),
              ('fc', {'version': 1}),
              ('dropout', {'version': 1})])}